# Mine releases from selected projects

This notebook mine the releases

In [1]:
import os
import datetime
import json
import re
import traceback
import math

from multiprocessing import Pool
import pandas as pd

from ipywidgets import IntProgress
from IPython.display import display

import releasy

In [2]:
from util import (
    DATA_PATH,
    REPO_PATH,
    TMP_PATH, 
    CPU,
    CycleType,
    RAPID_RELEASE_LIM,
    TRAD_RELEASE_LIM,
    delta2days,
    is_rapid_release,
    is_trad_release,
)

In [3]:
selected_projects = pd.read_csv(DATA_PATH / '10_projects_selected.csv')

## Mine Releases


In [4]:
def mine(name: str) -> releasy.Project:
    elapsed_time = datetime.datetime.now()
    
    try:
        repo_path = str(REPO_PATH / name)
        project = releasy.Miner(repo_path, name).apply(
            releasy.FinalReleaseMiner(),
            releasy.HistoryCommitMiner(),
            releasy.BaseReleaseMiner(),
            releasy.ContributorMiner(),
            releasy.SemanticReleaseMiner()
        ).mine()

        rapid_releases = [release for release in project.main_releases if is_rapid_release(release)]
        trad_releases = [release for release in project.main_releases if is_trad_release(release)]

        project_data = {
            'project': project.name,
            'prefixes': len(project.releases.prefixes()),
            'prefixes_names': str(" ".join(project.releases.prefixes())),
            'main_releases': len(project.main_releases),
            'rapid_releases': len(rapid_releases),
            'trad_releases': len(trad_releases),
            'patches': len(project.patches),
            'releases': len(project.releases)
        }
        
        release_data = []
        for srelease in project.main_releases:
            # release = srelease.release
            cycle_days = delta2days(srelease.cycle)
            delay = delta2days(srelease.delay)
            prev_release = srelease.prev_main_release

            if srelease.commits:
                duration = delta2days(srelease.time - srelease.commits.first(lambda c: c.author_time).author_time)
            else:
                duration = 0
                
            if srelease.commits and prev_release:
                commits_in_advance = [commit for commit in srelease.commits if commit.author_time < prev_release.time]
            else: 
                commits_in_advance = []
                
            
            release_data.append({
                'project': project.name,
                'release': srelease.name,
                'previous_release': srelease.prev_main_release.name if srelease.prev_main_release else '',
                'cycle': srelease.cycle,
                'cycle_days': cycle_days,
                # 'cycle_rdays': math.ceil(cycle_days),
                'rapid_release': is_rapid_release(srelease),
                'trad_release': is_trad_release(srelease),
                
                'commits': len(srelease.commits),
                'commits_in_advance': len(commits_in_advance),
                'start_delay': delay,
                'start_delay_rdays': math.ceil(delay),
                'duration': duration,
                # 'duration_rdays': math.ceil(duration),
                'patches': len(srelease.patches)     
            })
        
        elapsed_time = datetime.datetime.now() - elapsed_time
        project_data['time'] = elapsed_time
    except Exception as err:
        print(f"{name:40} {err=}")
        traceback.print_exception(err)
        project_data = {}
        release_data = []
    
    return project_data, release_data

In [5]:
project_names = list(selected_projects['project'])
# project_names = project_names[:1]

In [6]:
with Pool(processes=CPU) as pool:
    processed = 0
    project_results = []
    release_results = []
    progress = IntProgress(min=0, max=len(project_names))
    display(progress)
    for result in pool.imap_unordered(mine, project_names):
        project_result, release_result = result
        project_results.append(project_result)
        release_results.extend(release_result)
        progress.value += 1


IntProgress(value=0, max=1665)

In [7]:
projects = pd.DataFrame(project_results)
projects.sample(1)

,project,prefixes,prefixes_names,main_releases,rapid_releases,trad_releases,patches,releases,time
142,chainer/chainer,1,v,60,47,3,18,78,0 days 00:00:02.274215


In [8]:
releases = pd.DataFrame(release_results)
releases.sample(10)

,project,release,previous_release,cycle,cycle_days,rapid_release,trad_release,commits,commits_in_advance,start_delay,start_delay_rdays,duration,patches
24019,microsoft/Windows-universal-samples,108515,108514,0 days 00:00:00,0.000000,False,False,0,0,0.000000,0,0.000000,0
32967,abseil/abseil-cpp,20200225.2,20200225.1,48 days 22:45:31,48.948275,False,False,1,0,48.948275,49,0.000000,0
47422,psf/requests,v2.11.0,v2.10.0,100 days 16:55:09,100.704965,False,True,74,0,0.003843,1,100.701123,1
44124,chrislusf/seaweedfs,2.50,2.49,7 days 19:48:43,7.825498,True,False,34,4,-3.046134,-3,10.871632,0
13975,wix/react-native-calendars,1.151.0,1.150.0,0 days 15:13:02,0.634051,True,False,1,0,0.633252,1,0.000799,0
8435,concourse/concourse,v1.4.0,v1.3.0,23 days 02:02:25,23.085012,True,False,80,0,2.870891,3,20.214120,1
12825,QuantConnect/Lean,13365,13364,0 days 00:00:00,0.000000,False,False,0,0,0.000000,0,0.000000,0
46948,webpack/webpack,v5.51.0,v5.50.0,8 days 14:32:41,8.606030,True,False,14,0,3.664653,4,4.941377,2
42253,simdjson/simdjson,v0.9.0,v0.8.0,53 days 22:04:36,53.919861,False,False,101,0,2.389884,3,51.529977,7
25329,HabitRPG/habitica,v3.17.0,v3.16.0,4 days 19:04:09,4.794549,True,False,15,0,0.529479,1,4.265069,0


In [9]:
len(projects)

1665

In [10]:
len(releases)

52836

In [11]:
len(releases.query("rapid_release == True"))

29306

In [12]:
len(releases.query("trad_release == True"))

11296

In [13]:
len(releases.query("rapid_release == False and trad_release == False"))

12234

In [14]:
releases['commits'].sum()

5520108

In [15]:
projects.to_csv(DATA_PATH / '20_projects.csv', index=False)

In [16]:
releases.to_csv(DATA_PATH / '20_releases.csv', index=False)